In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

In [ ]:
# from scipy.sparse import csr_matrix
# #Creating a sparse matrix
# anime_pivot=pd.read_csv('csv/2020/anime_2020_pivot.csv')
# anime_user_matrix = csr_matrix(anime_pivot.values)

In [2]:
anime_data_cleaned = pd.read_csv('csv/2020/anime_2020_cleaned.csv')
rating_data = pd.read_csv('csv/2020/rating_2020.csv')

In [7]:
anime_name = pd.read_csv('csv/2020/anime_2020_name.csv')

In [3]:
counts = rating_data['user_id'].value_counts()
rating_data = rating_data[rating_data['user_id'].isin(counts[counts >= 200].index)]
rating_data.rename(columns={'anime_id':'MAL_ID'}, inplace=True)

In [4]:
anime_rating_data = anime_data_cleaned.merge(rating_data, on='MAL_ID', how='inner')
anime_ratings = anime_rating_data[['user_id', 'MAL_ID', 'rating']]

## Uesr-based collaborative filtering

In [5]:
from sklearn.model_selection import train_test_split

# Split the data into training and testing sets based on the users
train_data, test_data = train_test_split(rating_data, test_size=0.2, random_state=42)

In [20]:
from scipy.sparse import csr_matrix
anime_user_matrix = train_data.pivot_table(index='MAL_ID', columns='user_id', values='rating').fillna(0)
train_data_sparse = csr_matrix(anime_user_matrix.fillna(0).values)

In [26]:
from sklearn.cluster import KMeans

# Cluster the users based on the sparse matrix
kmeans = KMeans(n_clusters=10, random_state=42).fit(train_data_sparse)

# Add the cluster labels to the user-item matrix
anime_user_matrix['cluster'] = kmeans.labels_

In [40]:
from sklearn.metrics.pairwise import cosine_similarity
def predict_ratings_item_based(name):
    anime_id = anime_name[anime_name['Name'] == name]['MAL_ID'].values[0]

    # Get the user_ids that the anime has actually been rated
    user_ids = test_data.loc[test_data['MAL_ID'] == anime_id]['user_id'].values

    # Get the cluster of the anime (e.g., 5)
    anime_cluster = anime_user_matrix['cluster'].loc[anime_user_matrix['cluster'].index == anime_id].values[0]
    
    # Get the users in the same cluster (e.g., [1, 2, 3, 4, 5, 6, 7, 8, 9, 10])
    animes_in_cluster = anime_user_matrix[anime_user_matrix['cluster'] == anime_cluster].index
    
    # Get the ratings of the animes in the same cluster
    ratings_of_animes_in_cluster = train_data_sparse[anime_user_matrix.index.isin(animes_in_cluster)]
    
    row_index = anime_user_matrix.index.get_loc(anime_id)

    # Calculate the similarity between the user and the animes in the same cluster
    similarity = cosine_similarity(train_data_sparse[row_index], ratings_of_animes_in_cluster)

    # Get the ratings for the specified user_ids that exist in train_data_sparse
    # It is actually searching it from anime_user_matrix but its fine because train_data_sparse is a csr_matrix made from anime_user_matrix
    common_user_ids = set(user_ids).intersection(set(anime_user_matrix.index.values))
    common_user_ids = np.array(sorted(list(common_user_ids)))
    
    common_user_ids_idx  = [anime_user_matrix.columns.get_loc(user_id) for user_id in common_user_ids]
    
    # Use this in case if you want to get all the user_ids from train_data_sparse
    user_ids_idx  = [anime_user_matrix.columns.get_loc(user_id) for user_id in user_ids]

    # get index of users_in_cluster, common_user_ids
    animes_in_cluster_idx = [anime_user_matrix.index.get_loc(anime_id) for anime_id in animes_in_cluster]

    # Get the ratings for the specified user_ids
    user_ratings = np.asarray(train_data_sparse[animes_in_cluster_idx, :][:, user_ids_idx].todense())

    # Get the top-k similar animes
    top_similar_animes = np.argsort(similarity)[0][-1]
    
    # Calculate the predicted ratings for the specified user_ids
    predicted_ratings = user_ratings.dot(similarity[0][top_similar_animes]) / np.sum(np.abs(similarity[0][top_similar_animes]))
    
    return predicted_ratings[0]


In [44]:
predicted_ratings = predict_ratings_item_based('Naruto')
predicted_ratings, predicted_ratings.shape

(array([0., 0., 0., ..., 0., 0., 0.]), (11197,))

In [32]:
id = anime_name[anime_name['Name'] == 'Naruto']['MAL_ID'].values[0]
actual_ratings = test_data.loc[test_data['MAL_ID']==id]['rating'].values
actual_ratings, actual_ratings.shape

(array([10,  7,  6, ...,  9,  9,  9]), (11197,))

In [42]:
def calculate_mae(predicted_ratings, actual_ratings):
    return np.mean(np.abs(predicted_ratings - actual_ratings))

In [43]:
calculate_mae(predicted_ratings, actual_ratings)

7.611592390818969

## Calculate similarity between all animes

In [45]:
from sklearn.metrics.pairwise import cosine_similarity

# Calculate the cosine similarity between items
anime_similarity = cosine_similarity(anime_user_matrix)

In [ ]:
# Generate recommendations using the similarity scores
norm = np.abs(anime_similarity).sum(axis=1)
predicted_anime_ratings = anime_similarity.dot(anime_user_matrix.values) / norm[:, np.newaaxis]

In [15]:
name = 'Naruto'
# get MAL_ID
anime_name[anime_name['Name'] == name]['MAL_ID'].values[0]

20

In [18]:
train_data.loc[train_data['MAL_ID'] == 20]

,user_id,MAL_ID,rating
30537002,187850,20,7
12200520,74712,20,3
45917065,281746,20,9
12703863,77893,20,8
23094810,141908,20,8
...,...,...,...
10465284,64078,20,10
7597911,46581,20,7
6039979,37088,20,7
52115670,319617,20,8


In [21]:
row_index = anime_user_matrix.index.get_loc(20)

In [ ]:
# Write a function to find top k similar anime from anime_pivot
def recommend_item_base(anime_name, k=10):
    anime_index = anime_user_matrix.index.get_loc(anime_name)
    anime_indices = np.argsort(anime_similarity[anime_index])[-k:][::-1]
    similar_anime_ids = anime_user_matrix.index[anime_indices]
    return similar_anime_ids

In [ ]:
recommend_item_base('Naruto')